# WIDS 2022

In [ ]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
import lightgbm as lgb
import gc
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
df_train=pd.read_csv('/kaggle/input/widsdatathon2022/train.csv')

df_test=pd.read_csv('/kaggle/input/widsdatathon2022/test.csv')

In [ ]:
len(df_train)

In [ ]:
len(df_test)

In [ ]:
df_train.isnull().mean()

In [ ]:
df_train['site_eui'].describe()

In [ ]:
df_train['Year_Factor'].value_counts()

In [ ]:
df_test['Year_Factor'].value_counts()

In [ ]:
df_test['Year_Factor'].value_counts()

In [ ]:
df_test.isnull().mean()

In [ ]:
df_train

In [ ]:
df_test

In [ ]:
def entrena_lgb(data,test,features,categorical,target):

    kfold=GroupKFold(n_splits=6)


    i=1

    r=[]
    
    pred_test=np.zeros(len(test))

    importancias=pd.DataFrame()

    importancias['variable']=features
    
    
    cat_ind=[features.index(x) for x in categorical if x in features]
    
    dict_cat={}
    
    categorical_numerical = data[categorical].dropna().select_dtypes(include=np.number).columns.tolist()
    
    categorical_transform=[x for x in categorical if x not in categorical_numerical]
    
    for l in categorical_transform:
        le = preprocessing.LabelEncoder()
        le.fit(list(data[l].dropna())+list(test[l].dropna()))

        dict_cat[l]=le

        data.loc[~data[l].isnull(),l]=le.transform(data.loc[~data[l].isnull(),l])
        test.loc[~test[l].isnull(),l]=le.transform(test.loc[~test[l].isnull(),l])
        
        

    for train_index,test_index in kfold.split(data,data[target],data['Year_Factor']):

        lgb_data_train = lgb.Dataset(data.loc[train_index,features].values,data.loc[train_index,target].values)
        lgb_data_eval = lgb.Dataset(data.loc[test_index,features].values,data.loc[test_index,target].values, reference=lgb_data_train)

        params = {
            'task': 'train',
            'boosting_type': 'gbdt',
            'objective': 'regression',
            #'device_type':'gpu',
            'metric': { 'rmse'},
            'num_iterations':7000,
            'max_bin':5395,
            "max_depth":12,
            "num_leaves":52,
            'learning_rate': 0.8727,
            "min_child_samples": 100,
            'feature_fraction': 0.9,
            "bagging_freq":1,
            'bagging_fraction': 0.9,
            "lambda_l1":10,
            "lambda_l2":10,
            "scale_pos_weight":30,
            'min_data_per_group':500,
            'verbose': 1    
        }




        modelo = lgb.train(params,
                           lgb_data_train,
                           num_boost_round=99990,
                           valid_sets=lgb_data_eval,
                           early_stopping_rounds=140,
                           verbose_eval=250,
                           categorical_feature=cat_ind
                          )

        importancias['gain_'+str(i)]=modelo.feature_importance(importance_type="gain")


        data.loc[test_index,'estimator']=modelo.predict(data.loc[test_index,features].values, num_iteration=modelo.best_iteration)
        
        pred_test=pred_test+modelo.predict(test[features].values, num_iteration=modelo.best_iteration)

        print ("Fold_"+str(i))
        a= (mean_squared_error(data.loc[test_index,target],data.loc[test_index,'estimator']))**0.5
        r.append(a)
        print (a)
        print ("")

        i=i+1
        
    for l in categorical_transform:

            data.loc[~data[l].isnull(),l]=dict_cat[l].inverse_transform(data.loc[~data[l].isnull(),l].astype(int))
            
            test.loc[~test[l].isnull(),l]=dict_cat[l].inverse_transform(test.loc[~test[l].isnull(),l].astype(int))
            
    importancias["gain_avg"]=importancias[["gain_1","gain_2","gain_3","gain_4","gain_5"]].mean(axis=1)
    importancias=importancias.sort_values("gain_avg",ascending=False).reset_index(drop=True)
    
    pred_test=(pred_test/6)
    
    
    oof=(mean_squared_error(data[target],data['estimator']))**0.5
    
    print (oof)
    print ("mean: "+str(np.mean(np.array(r))))
    print ("std: "+str(np.std(np.array(r))))
    
    dict_resultados={}
    
    dict_resultados['importancias']=importancias
    
    dict_resultados['predicciones']=pred_test
    
    
    
    return dict_resultados

In [ ]:
no_usar=['site_eui','id']

target='site_eui'

categorical=['Year_Factor','State_Factor','building_class','facility_type']

features=[x for x in df_train.columns if x not in no_usar]
features


In [ ]:
dict_resultados=entrena_lgb(data=df_train,test=df_test,features=features,categorical=categorical,target=target)

In [ ]:
dict_resultados['importancias'].head(7)

In [ ]:
temp=dict_resultados['importancias']

features_selected=temp['variable'].tolist()[0:4]

In [ ]:
dict_resultados_2=entrena_lgb(data=df_train,test=df_test,features=features_selected,categorical=categorical,target=target)

In [ ]:
df_train.groupby([ 'State_Factor', 'building_class', 'facility_type',
       'floor_area', 'year_built', 'energy_star_rating', 'ELEVATION']).size()

In [ ]:
df_train.groupby([ 'State_Factor', 'building_class', 'facility_type',
       'floor_area', 'year_built', 'energy_star_rating', 'ELEVATION']).size().value_counts()

In [ ]:
df_train.groupby([ 'State_Factor', 'building_class', 'facility_type',
       'floor_area', 'year_built',  'ELEVATION']).size().value_counts()

In [ ]:
features_selected_2=list(features_selected)

features_selected_2.extend(['State_Factor'])

In [ ]:
dict_resultados_3=entrena_lgb(data=df_train,test=df_test,features=features_selected_2,categorical=categorical,target=target)

In [ ]:
"""
variables=['facility_type',
 'energy_star_rating',
 'year_built',
# 'floor_area',
 'State_Factor'
          ]

df_train['combination_variables']=df_train[variables].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
df_test['combination_variables']=df_test[variables].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

features_selected_3=list(features_selected_2)
features_selected_3.extend(['combination_variables','january_min_temp'])

categorical.extend(['combination_variables'])


dict_resultados_4=entrena_lgb(data=df_train,test=df_test,features=features_selected_3,categorical=categorical,target=target)

dict_resultados_4['importancias']
"""

In [ ]:
df_test['site_eui']=(dict_resultados_2['predicciones'].copy()+dict_resultados_3['predicciones'].copy())/2

In [ ]:
df_test[['id','site_eui']]

In [ ]:
df_test[['id','site_eui']].to_csv('Test_LightBGM(V1.8).csv',index=False)